In [ ]:
from flask_restful import Api, Resource
from flask import Flask, request

app = Flask(__name__)
api = Api(app)

In [ ]:
# 这边的类名是自己定义的
class receive_pic(Resource):
    def put(self):
        #接收二进制流保存为图片
        img = request.get_data()
        with open("test.png", "wb") as f:
            f.write(img)
        return 0
        
# 后面的路径可以自己定义
api.add_resource(receive_pic,'/test')

In [ ]:
if __name__ == '__main__':
    app_port = 7100
    app.run(host="0.0.0.0", port=app_port, debug=False)

In [ ]:
import socket
import numpy as np

tcp_server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

tcp_server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, True)

tcp_server.bind(("0.0.0.0", 8267))

tcp_server.listen(128)

tcp_client, tcp_client_address= tcp_server.accept()

print("client's IP and port: ",tcp_client_address)
print("client: ",tcp_client)

data = tcp_client.recv(1024)

data = data.decode(encoding='utf-8')
print("data received from client: ", data)

data = data.split(" ")

sensor_data=[]

for i in range(len(data)):
    if i==0:
        tmp=data[i][1:-1]
        tmp=float(tmp)
        sensor_data.append(tmp)
    else:
        tmp=data[i][:-1]
        tmp=float(tmp)
        sensor_data.append(tmp)
sensor_data = np.array(sensor_data)

send_data = "thanks client! Your message is received".encode(encoding = "utf-8")
tcp_client.send(send_data)

tcp_client.close()

In [ ]:
print(sensor_data)

In [ ]:
from PIL import Image
from torchvision import transforms
import torch
import numpy as np

device=torch.device("cpu")

img_original = Image.open("test.png")
transforms = transforms.Compose([
    transforms.PILToTensor(),
    transforms.Resize((224,224)), # 裁剪为224*224
    ])
img_original = transforms(img_original)
#img_original = img_original.reshape((1,-1))
img_original = img_original.to(dtype=torch.float32).to(device)

In [ ]:
'''
Rmean = torch.load("pt_files/Rmean.pt"); Rstd = torch.load("pt_files/Rstd.pt")
Gmean = torch.load("pt_files/Gmean.pt"); Gstd = torch.load("pt_files/Gstd.pt")
Bmean = torch.load("pt_files/Bmean.pt"); Bstd = torch.load("pt_files/Bstd.pt")

for i in range(224):
    for j in range(224):
        img[0][i][j] = (img_original[0][i][j]-Rmean)/Rstd
        
for i in range(224):
    for j in range(224):
        img[1][i][j] = (img_original[1][i][j]-Gmean)/Gstd

for i in range(224):
    for j in range(224):
        img[2][i][j] = (img_original[2][i][j]-Bmean)/Bstd
'''

In [ ]:
import numpy as np

mean = np.load("npy_files/mean.npy")
mean = mean.astype("float32")
std = np.load("npy_files/std.npy")
sensor_data = (sensor_data-mean)/std
sensor_data =torch.from_numpy(sensor_data).to(dtype=torch.float32).to(device)

In [ ]:
# the fc model

import torch
import torch.nn as nn
import torch.nn.functional as F

class FC(nn.Module):
    def __init__(self):
        super(FC, self).__init__()
        self.hidden1=nn.Sequential(
                nn.Linear(in_features=11,out_features=256,bias=True), # 20+5 / 20+11
                nn.Dropout(p = 0.2),
                nn.ReLU())
        self.hidden2=nn.Sequential(
                nn.Linear(in_features=256,out_features=128,bias=True),
                nn.ReLU())
        self.hidden3=nn.Sequential(
                nn.Linear(in_features=128,out_features=100,bias=True),
                nn.ReLU())
        self.hidden4=nn.Sequential(
                nn.Linear(in_features=100,out_features=10,bias=True),
                nn.ReLU())

    def forward(self,x):
        x=self.hidden1(x)
        x=self.hidden2(x)
        x=self.hidden3(x)
        output=self.hidden4(x)
        return output

In [ ]:
# resnet-18 

import torch
import torch.nn as nn
import torch.nn.functional as F

#定义残差块ResBlock

class ResBlock(nn.Module):
    def __init__(self, inchannel, outchannel, identity_downsample=None, stride=1):
        super(ResBlock, self).__init__()
        
        #这里定义了残差块内连续的2个卷积层
        self.conv1 = nn.Conv2d(inchannel,outchannel,kernel_size=3,stride=stride,padding=1)
        self.bn1 = nn.BatchNorm2d(outchannel)
        self.conv2 = nn.Conv2d(outchannel,outchannel,kernel_size=3,stride=1,padding=1)
        self.bn2 = nn.BatchNorm2d(outchannel)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
            
    def forward(self, x):
        identity = x
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        
        # if identity_downsample is not None as default, then:
        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)
        
        x = x + identity
        x = self.relu(x)
        
        return x

In [ ]:
class ResNet_18(nn.Module):
    
    def __init__(self, image_channels, num_classes):
        
        super(ResNet_18, self).__init__()
        # self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        #resnet layers
        self.layer1 = self.__make_layer(64, 64, stride=1)
        self.layer2 = self.__make_layer(64, 128, stride=2)
        self.layer3 = self.__make_layer(128, 256, stride=2)
        self.layer4 = self.__make_layer(256, 512, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)
        
    def identity_downsample(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1), 
            nn.BatchNorm2d(out_channels)
        )  
    
    def __make_layer(self, in_channels, out_channels, stride):
        
        identity_downsample = None #默认是none,即identity-free shortcut
        if stride != 1:
            identity_downsample = self.identity_downsample(in_channels, out_channels)
        #对于64-128.128-256.256-512的第一个block,有stride=2,且outchannel=2*inchannel；
        #其他的block,64-64的全部2个,64-128的第2个，128-256的第2个，256-512的第2个，都是outchannel=inchannel
            
        return nn.Sequential(
            ResBlock(in_channels, out_channels, identity_downsample=identity_downsample, stride=stride), 
            ResBlock(out_channels, out_channels)
        )
    
    def forward(self, x):
        x = x.reshape((-1, 3, 224, 224)) # 变成1*3*224*224
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x 
    

In [ ]:
sensor_model = torch.load("moving_sensor_new.pth",map_location=torch.device('cpu'))
visual_model = torch.load("moving_visual_new.pth",map_location=torch.device('cpu'))

In [ ]:
weight1=0.2
weight2=0.8

with torch.no_grad():
    output1 = F.softmax(sensor_model(sensor_data)) ; print(output1) #触觉

    output2 = F.softmax(visual_model(img_original)) ; print(output2) #视觉

    outputs = output1*weight1+output2*weight2 ; #print(outputs)

    _, predicted = torch.max(outputs.data, 1)


In [ ]:
print(predicted)

In [ ]:
import socket
import sys
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host='0.0.0.0'
port=6000
s.bind((host,port))
s.listen(5)
msg = chr(predicted+97)
while True:
    clientsocket, addr = s.accept()
    print("连接地址: %s" % str(addr))
    clientsocket.send(msg.encode('utf-8'))
    clientsocket.close()